# Predictor: Random Forest Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

In [33]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

## Training

### Copy the data locally

In [34]:
from pathlib import Path

# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_DIR = Path('data')
DATA_FILE = DATA_DIR/'OxCGRT_latest.csv'
MODELS_DIR = Path('models')
VALID_DATA_DIR = Path('../../validation/data')

In [35]:
import os
import urllib.request

if not DATA_DIR.exists():
    DATA_DIR.mkdir()
    
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

(PosixPath('data/OxCGRT_latest.csv'), <http.client.HTTPMessage at 0x15cd3eed0>)

In [36]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [37]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [38]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-10-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [39]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [40]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [41]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols].copy()

In [42]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [43]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [44]:
df.describe()

,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
count,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000,80825.000000
mean,751.508729,1.834692,1.315509,1.262270,2.198985,0.560309,0.890603,0.930801,2.401695,1.634222,1.465487,1.213003,1.582938
std,4101.050935,1.274829,1.076823,0.882193,1.694078,0.720439,0.874306,0.890329,1.428798,0.734149,0.978778,0.799146,1.522393
min,-10034.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,1.000000,0.000000
50%,16.000000,2.000000,2.000000,2.000000,3.000000,0.000000,1.000000,1.000000,3.000000,2.000000,1.000000,1.000000,1.000000
75%,318.000000,3.000000,2.000000,2.000000,4.000000,1.000000,2.000000,2.000000,4.000000,2.000000,2.000000,2.000000,3.000000
max,123912.000000,3.000000,3.000000,2.000000,4.000000,2.000000,3.000000,2.000000,4.000000,2.000000,3.000000,2.000000,4.000000


In [45]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [46]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [47]:
X_samples.shape

(72610, 390)

In [48]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [49]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = RandomForestRegressor(max_depth=4, random_state=44)
# Fit model
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, random_state=44)

In [50]:
# Evaluate model
train_preds = model.predict(X_train)
# train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
# test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 245.0774667225245
Test MAE: 266.5755280164302


In [65]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.feature_importances_)):
    if coeff != 0.:
        print(col, coeff)

print(f'Num. estimators {model.n_estimators}')

Day -30 NewCases 0.00017689436794132233
Day -29 NewCases 2.9249339540177184e-05
Day -28 NewCases 1.5013453880910846e-05
Day -27 NewCases 0.0002643663142627273
Day -26 NewCases 0.00011863339633928796
Day -25 NewCases 0.00045255695145051143
Day -24 NewCases 9.797828199794835e-05
Day -23 NewCases 4.2162026011593474e-05
Day -22 NewCases 5.0511327638485486e-05
Day -21 NewCases 3.923540117278359e-05
Day -20 NewCases 0.0001531803246769631
Day -19 NewCases 0.0001277342146205009
Day -18 NewCases 0.00011017058164219395
Day -17 NewCases 0.0001833474545566282
Day -16 NewCases 0.0002402325485081367
Day -15 NewCases 9.089228184639465e-05
Day -14 NewCases 0.000114658657863033
Day -13 NewCases 0.03781911752743153
Day -12 NewCases 5.093233526050744e-05
Day -11 NewCases 4.117440440219768e-05
Day -10 NewCases 0.00020661871186458805
Day -9 NewCases 7.212918702641058e-05
Day -8 NewCases 0.00023877256818672905
Day -7 NewCases 0.001435889185371452
Day -6 NewCases 0.8984358309410109
Day -5 NewCases 0.03303910

In [52]:
# Save model to file
if not MODELS_DIR.exists():
    MODELS_DIR.mkdir()
with open(MODELS_DIR/'model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [53]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [54]:
# export ips file for the corresponding dates
hist_ips_df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
hist_ips_df = df[id_cols + npi_cols]
hist_ips_df.to_csv(VALID_DATA_DIR/"2020-12-2_historical_ip.csv")

In [55]:
%%time
preds_df = predict_df("2020-10-01", "2020-10-31", path_to_ips_file=VALID_DATA_DIR/"2020-12-2_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-10-01: 85.28790553019412
2020-10-02: 85.28790553019412
2020-10-03: 85.28790553019412
2020-10-04: 85.28790553019412
2020-10-05: 85.28790553019412
2020-10-06: 85.28790553019412
2020-10-07: 85.28790553019412
2020-10-08: 85.28790553019412
2020-10-09: 85.28790553019412
2020-10-10: 85.28790553019412
2020-10-11: 85.28790553019412
2020-10-12: 85.28790553019412
2020-10-13: 85.28790553019412
2020-10-14: 85.28790553019412
2020-10-15: 85.28790553019412
2020-10-16: 85.28790553019412
2020-10-17: 85.28790553019412
2020-10-18: 85.28790553019412
2020-10-19: 85.28790553019412
2020-10-20: 85.28790553019412
2020-10-21: 85.28790553019412
2020-10-22: 85.28790553019412
2020-10-23: 85.28790553019412
2020-10-24: 85.28790553019412
2020-10-25: 85.28790553019412
2020-10-26: 85.28790553019412
2020-10-27: 85.28790553019412
2020-10-28: 85.28790553019412
2020-10-29: 85.28790553019412
2020-10-30: 85.28790553019412
2020-10-31: 85.28790553019412

Predicting for Afghanistan__nan
2020-10-01

2020-10-24: 1821.3851361510792
2020-10-25: 1821.3851361510792
2020-10-26: 1821.3851361510792
2020-10-27: 1821.3851361510792
2020-10-28: 1821.3851361510792
2020-10-29: 1821.3851361510792
2020-10-30: 1821.3851361510792
2020-10-31: 1821.3851361510792

Predicting for Azerbaijan__nan
2020-10-01: 2813.2892674657282
2020-10-02: 3237.990517124838
2020-10-03: 3311.7683754387526
2020-10-04: 3991.5418380629258
2020-10-05: 3471.065094035387
2020-10-06: 1762.19342746501
2020-10-07: 2022.5796665230102
2020-10-08: 1897.1703405951307
2020-10-09: 1897.1703405951307
2020-10-10: 2439.7423768400804
2020-10-11: 2174.620311329042
2020-10-12: 1836.1128773384926
2020-10-13: 1821.3851361510792
2020-10-14: 1821.3851361510792
2020-10-15: 1821.3851361510792
2020-10-16: 1821.3851361510792
2020-10-17: 1821.3851361510792
2020-10-18: 1821.3851361510792
2020-10-19: 1821.3851361510792
2020-10-20: 1821.3851361510792
2020-10-21: 1821.3851361510792
2020-10-22: 1821.3851361510792
2020-10-23: 1821.3851361510792
2020-10-24: 

2020-10-17: 85.28790553019412
2020-10-18: 85.28790553019412
2020-10-19: 85.28790553019412
2020-10-20: 85.28790553019412
2020-10-21: 85.28790553019412
2020-10-22: 85.28790553019412
2020-10-23: 85.28790553019412
2020-10-24: 85.28790553019412
2020-10-25: 85.28790553019412
2020-10-26: 85.28790553019412
2020-10-27: 85.28790553019412
2020-10-28: 85.28790553019412
2020-10-29: 85.28790553019412
2020-10-30: 85.28790553019412
2020-10-31: 85.28790553019412

Predicting for Bosnia and Herzegovina__nan
2020-10-01: 621.3256944117101
2020-10-02: 672.125222000233
2020-10-03: 758.5244037328935
2020-10-04: 1143.1593155726807
2020-10-05: 1082.6084995499655
2020-10-06: 477.8231879328562
2020-10-07: 715.6689834403492
2020-10-08: 715.6689834403492
2020-10-09: 792.0680035534849
2020-10-10: 1075.906375438138
2020-10-11: 977.1507553050419
2020-10-12: 733.2098676273475
2020-10-13: 715.6689834403492
2020-10-14: 722.0175247168621
2020-10-15: 783.1622013633746
2020-10-16: 1027.103089041069
2020-10-17: 870.767935954

2020-10-06: 85.28790553019412
2020-10-07: 85.28790553019412
2020-10-08: 85.28790553019412
2020-10-09: 85.28790553019412
2020-10-10: 85.28790553019412
2020-10-11: 85.28790553019412
2020-10-12: 85.28790553019412
2020-10-13: 85.28790553019412
2020-10-14: 85.28790553019412
2020-10-15: 85.28790553019412
2020-10-16: 85.28790553019412
2020-10-17: 85.28790553019412
2020-10-18: 85.28790553019412
2020-10-19: 85.28790553019412
2020-10-20: 85.28790553019412
2020-10-21: 85.28790553019412
2020-10-22: 85.28790553019412
2020-10-23: 85.28790553019412
2020-10-24: 85.28790553019412
2020-10-25: 85.28790553019412
2020-10-26: 85.28790553019412
2020-10-27: 85.28790553019412
2020-10-28: 85.28790553019412
2020-10-29: 85.28790553019412
2020-10-30: 85.28790553019412
2020-10-31: 85.28790553019412

Predicting for Brazil__Bahia
2020-10-01: 1751.9386354420785
2020-10-02: 2387.883331352215
2020-10-03: 1928.5797568558214
2020-10-04: 1877.4332720325897
2020-10-05: 1923.4877863361128
2020-10-06: 1235.5774746265326
2020-

2020-10-25: 715.6689834403492
2020-10-26: 830.1283712506098
2020-10-27: 1293.1613208732927
2020-10-28: 1472.8651941131957
2020-10-29: 1246.771891947424
2020-10-30: 748.7578457962435
2020-10-31: 722.0175247168621

Predicting for Brazil__Mato Grosso
2020-10-01: 208.90328481731922
2020-10-02: 104.4461095402173
2020-10-03: 372.1506657992628
2020-10-04: 530.495067633053
2020-10-05: 717.1656698862039
2020-10-06: 428.8062231712804
2020-10-07: 411.8643844927576
2020-10-08: 387.16224775382926
2020-10-09: 374.7523630967641
2020-10-10: 509.28562425249874
2020-10-11: 694.7493595866127
2020-10-12: 682.969192540798
2020-10-13: 604.8571741806612
2020-10-14: 559.1259928613104
2020-10-15: 500.3327269890828
2020-10-16: 715.6689834403492
2020-10-17: 715.6689834403492
2020-10-18: 715.6689834403492
2020-10-19: 715.6689834403492
2020-10-20: 715.6689834403492
2020-10-21: 715.6689834403492
2020-10-22: 715.6689834403492
2020-10-23: 715.6689834403492
2020-10-24: 715.6689834403492
2020-10-25: 715.6689834403492
2

...


Predicting for United States__Louisiana
2020-10-01: 593.8668946699333
2020-10-02: 1927.9002280286734
2020-10-03: 1377.3246333468821
2020-10-04: 1768.5613779128346
2020-10-05: 2592.7602439365232
2020-10-06: 856.4224477188935
2020-10-07: 1002.0787619633811
2020-10-08: 1502.5669736065302
2020-10-09: 1734.8892975381468
2020-10-10: 1860.0280221773503
2020-10-11: 1556.8718770734556
2020-10-12: 1032.2088921954214
2020-10-13: 1145.9594316487037
2020-10-14: 1589.8793295060307
2020-10-15: 1805.8371579821833
2020-10-16: 1821.3851361510792
2020-10-17: 1618.0165537199684
2020-10-18: 1212.132782631878
2020-10-19: 1322.0118571767002
2020-10-20: 1589.8793295060307
2020-10-21: 1805.8371579821833
2020-10-22: 1821.3851361510792
2020-10-23: 1664.9827236072035
2020-10-24: 1466.1139386342204
2020-10-25: 1581.3587436931641
2020-10-26: 1821.3851361510792
2020-10-27: 1821.3851361510792
2020-10-28: 1821.3851361510792
2020-10-29: 1810.1668533226996
2020-10-30: 1821.3851361510792
2020-10-31: 1821.3851361510792



2020-10-25: 715.6689834403492
2020-10-26: 715.6689834403492
2020-10-27: 715.6689834403492
2020-10-28: 715.6689834403492
2020-10-29: 715.6689834403492
2020-10-30: 715.6689834403492
2020-10-31: 715.6689834403492

Predicting for United States__North Carolina
2020-10-01: 5902.367640922385
2020-10-02: 5982.779687029691
2020-10-03: 4091.806136159989
2020-10-04: 4824.35038623419
2020-10-05: 5861.919722063389
2020-10-06: 1785.5915492335562
2020-10-07: 5926.619328987279
2020-10-08: 5888.428059265195
2020-10-09: 3822.761459882955
2020-10-10: 5562.28226914896
2020-10-11: 5888.428059265195
2020-10-12: 3440.264676502741
2020-10-13: 5952.809051281515
2020-10-14: 5888.428059265195
2020-10-15: 3837.8798211333387
2020-10-16: 5952.809051281515
2020-10-17: 5888.428059265195
2020-10-18: 3588.8774442428758
2020-10-19: 5952.809051281515
2020-10-20: 5888.428059265195
2020-10-21: 3867.368818323065
2020-10-22: 5952.809051281515
2020-10-23: 5888.428059265195
2020-10-24: 3604.0859784682025
2020-10-25: 5952.80905

2020-10-10: 18331.065508848915
2020-10-11: 11458.674746304512
2020-10-12: 3440.264676502741
2020-10-13: 7671.565475656322
2020-10-14: 11338.303619280161
2020-10-15: 15126.644794828195
2020-10-16: 14410.041874654024
2020-10-17: 11458.674746304512
2020-10-18: 3892.052486868494
2020-10-19: 8657.128592650952
2020-10-20: 11926.754586162595
2020-10-21: 14547.93830891594
2020-10-22: 12962.747509368206
2020-10-23: 11228.835765943455
2020-10-24: 4839.015007707621
2020-10-25: 8814.48970556606
2020-10-26: 11681.882574605614
2020-10-27: 13901.996831176442
2020-10-28: 12320.009723653226
2020-10-29: 10842.927214738018
2020-10-30: 9716.02409290397
2020-10-31: 11432.321736604184

Predicting for United States__Oklahoma
2020-10-01: 2668.1428729753075
2020-10-02: 2409.7649097563162
2020-10-03: 1852.79455241177
2020-10-04: 1821.3851361510792
2020-10-05: 1550.5233357969423
2020-10-06: 1252.982724481772
2020-10-07: 1605.4273076749266
2020-10-08: 1805.8371579821833
2020-10-09: 1821.3851361510792
2020-10-10: 

2020-10-06: 1235.5774746265326
2020-10-07: 1605.4273076749266
2020-10-08: 1805.8371579821833
2020-10-09: 1821.3851361510792
2020-10-10: 1821.3851361510792
2020-10-11: 1684.3100162179203
2020-10-12: 1563.5265999069034
2020-10-13: 1821.3851361510792
2020-10-14: 1821.3851361510792
2020-10-15: 1821.3851361510792
2020-10-16: 1821.3851361510792
2020-10-17: 1821.3851361510792
2020-10-18: 1821.3851361510792
2020-10-19: 1821.3851361510792
2020-10-20: 1821.3851361510792
2020-10-21: 1821.3851361510792
2020-10-22: 1821.3851361510792
2020-10-23: 1821.3851361510792
2020-10-24: 1821.3851361510792
2020-10-25: 1821.3851361510792
2020-10-26: 1821.3851361510792
2020-10-27: 1821.3851361510792
2020-10-28: 1821.3851361510792
2020-10-29: 1821.3851361510792
2020-10-30: 1821.3851361510792
2020-10-31: 1821.3851361510792

Predicting for United States__Virginia
2020-10-01: 2222.932969503962
2020-10-02: 2411.125484447031
2020-10-03: 2426.673462615927
2020-10-04: 2425.544602214197
2020-10-05: 2629.051659844369
2020

2020-10-04: 459.59724727407587
2020-10-05: 675.5911555765895
2020-10-06: 428.8062231712804
2020-10-07: 411.8643844927576
2020-10-08: 387.16224775382926
2020-10-09: 374.7523630967641
2020-10-10: 509.28562425249874
2020-10-11: 694.7493595866127
2020-10-12: 682.969192540798
2020-10-13: 604.8571741806612
2020-10-14: 559.1259928613104
2020-10-15: 500.3327269890828
2020-10-16: 715.6689834403492
2020-10-17: 715.6689834403492
2020-10-18: 715.6689834403492
2020-10-19: 715.6689834403492
2020-10-20: 715.6689834403492
2020-10-21: 715.6689834403492
2020-10-22: 715.6689834403492
2020-10-23: 715.6689834403492
2020-10-24: 715.6689834403492
2020-10-25: 715.6689834403492
2020-10-26: 715.6689834403492
2020-10-27: 715.6689834403492
2020-10-28: 715.6689834403492
2020-10-29: 715.6689834403492
2020-10-30: 715.6689834403492
2020-10-31: 715.6689834403492

Predicting for United States Virgin Islands__nan
2020-10-01: 85.28790553019412
2020-10-02: 85.28790553019412
2020-10-03: 85.28790553019412
2020-10-04: 85.287

In [56]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
274,Aruba,NaN,2020-10-01,85.287906
275,Aruba,NaN,2020-10-02,85.287906
276,Aruba,NaN,2020-10-03,85.287906
277,Aruba,NaN,2020-10-04,85.287906
278,Aruba,NaN,2020-10-05,85.287906


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [57]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip {VALID_DATA_DIR}/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [58]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,85.28790553019412
Aruba,,2020-08-02,85.28790553019412
Aruba,,2020-08-03,85.28790553019412
Aruba,,2020-08-04,85.28790553019412
Afghanistan,,2020-08-01,85.28790553019412
Afghanistan,,2020-08-02,85.28790553019412
Afghanistan,,2020-08-03,85.28790553019412
Afghanistan,,2020-08-04,85.28790553019412
Angola,,2020-08-01,85.28790553019412


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [59]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [60]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file=VALID_DATA_DIR/"2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [61]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file=VALID_DATA_DIR/"future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 51.9 ms, sys: 22.3 ms, total: 74.2 ms
Wall time: 2.84 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [62]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-17
End date: 2021-06-15


In [63]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
# DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [64]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-17 to 2021-06-15...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 5.64 s, sys: 1.44 s, total: 7.08 s
Wall time: 7min 27s
